Hello, let me explained this chatbot. 

Why i'm rename Simple Chatbot? because this ChatBot just only using Clasification Concept. We have list of input text conversation and output conversation. This chatbot using data history in dataset, so if this Chatbot looking sentences not in training, it cann't detection. I'm not sure this chatbot can be responses using new data from user, but if we want use like that, we can using Generate Chatbot. This my argument has I know, but if you have more knowladge, let's discussion soon. 

In [4]:
#load data
import json
import pandas as pd

with open('src/humanText.json') as data_file:
    data = json.load(data_file)

In [9]:
textInput = []
humanText = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        textInput.append(pattern)
        humanText.append(intent['tag'])

df = pd.DataFrame({'textInput': textInput,
                    'intents': humanText})
df.head()

,textInput,intents
0,Halo Ara,salam
1,Halo,salam
2,Halo,salam
3,Selamat Pagi,salam
4,Selamat Siang,salam


In [10]:
df.intents.value_counts()

laporOrang            27
hubungan              15
laporDiri             11
bye                   11
pendidikanKorban      10
kronologi              9
salam                  8
pekerjaan              8
usiaPelaku             7
korbanLain             7
adaKorbanLain          5
pendidikanPelaku       4
tidakadaKorbanLain     3
Name: intents, dtype: int64

# Pre Processing

In [11]:
# data cleansing
import string

# convert lowercase
df.textInput = df.textInput.apply(lambda x: x.lower())

# remove punctuation
exclude = set(string.punctuation)
df.textInput = df.textInput.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

### Label Encoding to labelling tag category

In [12]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df.intents)
y_train = to_categorical(y_train)

In [14]:
all_vocab = []
length = []

for idx, row in df.iterrows():
    sent = row['textInput']
    [all_vocab.append(i) for i in sent.split()]
    length.append(len(sent.split()))

In [15]:
len(all_vocab)

1038

In [16]:
max(length)

152

In [17]:
len(set(all_vocab))

463

### Vectorization to convert sentences be vector

In [20]:
from tensorflow.keras.layers import TextVectorization

max_vocab_length = 462
max_length = 152

text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       standardize='lower_and_strip_punctuation',
                                       split='whitespace',
                                       ngrams=None,
                                       output_mode='int',
                                       output_sequence_length=max_length
                                       )

In [21]:
text_vectorization.adapt(df.textInput)
#text_vectorization.get_vocabulary()
#text_vectorization('halo ara ingin membantu')

### Embedding to create arrary from sentences

In [22]:
from tensorflow.keras.layers import Embedding
embedding = Embedding(input_dim=max_vocab_length,
                      output_dim=16,
                      embeddings_initializer="uniform",
                      input_length=max_length)
import numpy as np
res_embed = embedding(np.array([[71, 17,  7, 13,  0,  0]]))
res_embed

<tf.Tensor: shape=(1, 6, 16), dtype=float32, numpy=
array([[[-0.01043533, -0.02808155, -0.0014995 ,  0.0189749 ,
         -0.02920434, -0.02809837,  0.04671493, -0.04955262,
         -0.02088817, -0.01756253, -0.02715209, -0.00880202,
         -0.01827861,  0.02789671, -0.03300985, -0.04152896],
        [ 0.00759599,  0.02075017,  0.03771547, -0.0112205 ,
         -0.0495244 , -0.02746574, -0.01428571,  0.0465991 ,
         -0.03107158, -0.03881729, -0.03719347, -0.03745774,
         -0.0485856 ,  0.01656837, -0.0347409 ,  0.03383237],
        [ 0.0127707 ,  0.00787083,  0.00541034,  0.00498069,
         -0.0037182 ,  0.04617606, -0.02310144, -0.02322035,
          0.0345713 , -0.03708357,  0.02578181,  0.03070692,
         -0.03388425,  0.03360916,  0.01410476,  0.03995189],
        [-0.00442473, -0.00228377, -0.04782085,  0.02993448,
          0.00351559,  0.02216125, -0.01903405,  0.03766126,
         -0.04237236,  0.04690632,  0.00026895,  0.04669721,
         -0.00195068,  0.00747

# Modelling

### Neural Network

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

l0 = Dense(units=1, input_shape=[1])
model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 3.1057
Epoch 2/500
1/1 [==============================] - 0s 16ms/step - loss: 2.6004
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 2.1996
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 1.8811
Epoch 5/500
1/1 [==============================] - 0s 8ms/step - loss: 1.6274
Epoch 6/500
1/1 [==============================] - 0s 8ms/step - loss: 1.4248
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 1.2625
Epoch 8/500
1/1 [==============================] - 0s 8ms/step - loss: 1.1318
Epoch 9/500
1/1 [==============================] - 0s 8ms/step - loss: 1.0262
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 0.9403
Epoch 11/500
1/1 [==============================] - 0s 16ms/step - loss: 0.8700
Epoch 12/500
1/1 [==============================] - 0s 16ms/step - loss: 0.8120
Epoch 13/500
1/1 [==============================] - 0s 8ms/

In [14]:
print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))

[[18.985153]]
Here is what I learned: []


In [ ]:
import tensorflow as tf
data = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = data.load_data()

training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation=tf.nn.relu),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

model.compile(optimizer='adam',
       loss='sparse_categorical_crossentropy',
       metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

### LSTM

In [26]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, LSTM
inputs = Input(shape=(1,), dtype='string')
x = text_vectorization(inputs)
x = embedding(x)
x = LSTM(12)(x)
outputs = Dense(13, activation='softmax')(x)
modelLSTM = Model(inputs, outputs, name="LSTM_model")

In [27]:
# compile model
modelLSTM.compile(loss='categorical_crossentropy',
                   optimizer='adam',
                   metrics=["accuracy"])

In [28]:
modelLSTM.fit(df.textInput,
               y_train,
               epochs=150,
               verbose=0)

In [116]:
modelLSTM.evaluate(df.textInput, y_train)

2/2 [==============================] - 0s 7ms/step - loss: 0.2745 - accuracy: 1.0000


[0.27448540925979614, 1.0]

In [118]:
modelLSTM.save("modelLSTM.tf")

INFO:tensorflow:Assets written to: modelLSTM.tf\assets


INFO:tensorflow:Assets written to: modelLSTM.tf\assets


In [119]:
import pickle
le_filename = open("label_encoder.pickle", "wb")
pickle.dump(le, le_filename)
le_filename.close()

### Neural Network